In [ ]:
import pandas as pd
from pathlib import Path

f_stats = Path("npbstats_pitching_with_stint.csv")
f_jp  = Path("npb.jp_Pitching_with_stint.csv")
f_people = Path("npb_People_combined_cleaned.csv")
out_file = Path("#######3.csv")

stats  = pd.read_csv(f_stats)
jp = pd.read_csv(f_jp)
people = pd.read_csv(f_people)


C:\Users\Acer\AppData\Local\Temp\ipykernel_22280\1320520900.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  stats  = pd.read_csv(f_stats)


In [ ]:

def clean_ids(s: pd.Series) -> pd.Series:
    return (s.astype("string").str.strip().str.replace(r"\.0$", "", regex=True).replace({"nan": pd.NA}))

for df, col in [
    (stats, "SearchID"), (stats, "Player ID"),
    (jp, "Player ID"),
    (people,"SearchID"), (people,"Player ID")]:
    if col in df.columns:
        df[col] = clean_ids(df[col])

jp["SearchID"] = jp.get("SearchID", pd.Series(pd.NA, index=jp.index))
stats["Player ID"] = stats.get("Player ID", pd.Series(pd.NA, index=stats.index))

pid2sid = (people.dropna(subset=["Player ID","SearchID"]).drop_duplicates("Player ID").set_index("Player ID")["SearchID"])
sid2pid = (people.dropna(subset=["SearchID","Player ID"]).drop_duplicates("SearchID").set_index("SearchID")["Player ID"])

jp["SearchID"] = jp["SearchID"].fillna(jp["Player ID"].map(pid2sid))
stats["Player ID"] = stats["Player ID"].fillna(stats["SearchID"].map(sid2pid))

for df in (stats, jp):
    df["Team"] = df["Team"].astype("string").str.upper().str.strip()

metric_cols = ["G","W","L","BF","BB","SV","CG","H","HR","R","SO"]


In [ ]:
def norm_num(col: pd.Series) -> pd.Series:
    num = pd.to_numeric(col, errors="coerce")
    return num.apply(lambda x: "" if pd.isna(x) else str(int(x)))

def make_key(df: pd.DataFrame) -> pd.Series:
    cid = df["SearchID"].fillna(df["Player ID"]).astype("string").str.zfill(7)
    yr_team_st = (df["Year"].astype("string") + "|" + df["Team"] + "|" +norm_num(df["stint"]))
    metrics = (df[metric_cols].apply(norm_num).agg("|".join, axis=1))
    return cid + "|" + yr_team_st + "|" + metrics

In [ ]:
stats["__key__"] = make_key(stats)
jp["__key__"] = make_key(jp)

stats = stats.drop_duplicates("__key__", keep="first")
jp = jp.drop_duplicates("__key__", keep="first")
jp = jp.loc[~jp["__key__"].isin(set(stats["__key__"]))]

merged = pd.concat([stats, jp], ignore_index=True).drop(columns="__key__")

need_sid = merged["SearchID"].isna()
merged.loc[need_sid,"SearchID"] = merged.loc[need_sid,"Player ID"].map(pid2sid)

In [ ]:
merged["__sid_num"] = pd.to_numeric(merged["SearchID"], errors="coerce")
merged = (merged.sort_values(["__sid_num","Year","Team","stint"]).drop(columns="__sid_num").reset_index(drop=True))
out_file = Path("npb_pitchin@@@@@@@@@@@@@@@@@@@@@@@@eee331322131231.csv")
merged.to_csv(out_file, index=False)

In [ ]:
import pandas as pd
from pathlib import Path

merged_file = Path("npb_pitchin@@@@@@@@@@@@@@@@@@@@@@@@eee331322131231.csv")
df = pd.read_csv(merged_file)

metric_cols = ["G","W","L","BF","BB","SV","CG","H","HR","R","SO"]
key_cols = ["SearchID","Year"] + metric_cols

for col in key_cols:
    df[col] = (pd.to_numeric(df[col], errors="ignore").astype("string").str.strip().str.replace(r"\.0$", "", regex=True))

df["__has_rates__"] = df["K%"].notna() & df["BB%"].notna() & (df["K%"]!="") & (df["BB%"]!="")


to_drop = []
to_update_stint = []

for _, dup in df.groupby(key_cols):
    if len(dup) < 2:
        continue
    with_rates = dup[dup["__has_rates__"]]
    without_rates = dup[~dup["__has_rates__"]]
    if len(with_rates) == 1 and len(without_rates) >= 1:
        keep_idx = with_rates.index[0]
        drop_idx = without_rates.index.tolist()
        to_update_stint.append(keep_idx)
        to_drop.extend(drop_idx)

df.loc[to_update_stint, "stint"] = 2
df = df.drop(index=to_drop).reset_index(drop=True)
df = df.drop(columns="__has_rates__")

out_file = merged_file.with_name("npb_pitching_merged_finaeee331322131231@@@.csv")
df.to_csv(out_file, index=False)

In [ ]:
import pandas as pd
from pathlib import Path

INPUT_FILE  = Path("npb_pitching_merged_finaeee331322131231@@@.csv")
OUTPUT_FILE = INPUT_FILE.with_name(INPUT_FILE.stem + "all_filled.csv")

df = pd.read_csv(INPUT_FILE, low_memory=False)

team2league = (df[df["League"].isin(["CL", "PL"])].groupby(["Team", "League"]).size().unstack(fill_value=0).idxmax(axis=1).to_dict())

blank_mask = df["League"].isna() | (df["League"].astype(str).str.strip() == "")
df.loc[blank_mask, "League"] = df.loc[blank_mask, "Team"].map(team2league)

still_blank = df["League"].isna() | (df["League"].astype(str).str.strip() == "")

df.to_csv(OUTPUT_FILE, index=False)